In [1]:
import os

import numpy as np
import cupy as cp
import glob

import cudf
import nvtabular as nvt
from nvtabular import ColumnSelector

In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import cudf
import cupy
import nvtabular as nvt
import json

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
INPUT_DIR = "/recsys/coveo_task1_v42"

In [ ]:
data = cudf.read_parquet(os.path.join(INPUT_DIR, 'merged_session_table.parquet'))
data.head()

In [ ]:
#data.dtypes

In [ ]:
#data.shape

In [5]:
data1 = pd.read_parquet(os.path.join(INPUT_DIR, 'merged_session_table.parquet'))
#data1.head()

In [6]:
data1.head()

,session_id_hash,day_index,product_sku_hash_count,product_sku_hash_list_seq,product_url_hash_count,product_url_hash_list_seq,has_been_removed_from_cart-list_seq,has_been_added_to_cart-list_seq,has_been_purchased-list_seq,has_been_detailed-list_seq,has_been_clicked-list_seq,event_type-list_seq,product_action_filled-list_seq,nb_interactions-list_seq,category_hash-list_seq,main_category-list_seq,price_bucket-list_seq,mean_price_hierarchy-list_seq,mean_price_main-list_seq,ts-first,ts-last,ts-list_seq,timestamp_hour_cos-list_seq,timestamp_hour_sin-list_seq,timestamp_wd_sin-list_seq,timestamp_wd_cos-list_seq,timestamp_age_days-list_seq,timestamp_age_days_norm-list_seq,original_session_id_hash,flat_query_vector,flat_product_skus_hash,flat_clicked_skus_hash,impressions_size,clicks_size,nb_queries,clicked-flag
0,64548,42,28,"[2278, 1, 1, 1, 352, 1, 1, 2911, 1, 3184, 1, 1...",28,"[4621, 2010, 10385, 68, 864, 1, 9252, 5913, 36...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, ...","[2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, ...","[-0.17000507, -0.17000507, -0.17000507, -0.170...","[4, 0, 0, 0, 4, 0, 0, 4, 0, 4, 0, 0, 4, 0, 4, ...","[1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, ...","[3, 0, 0, 0, 1, 0, 0, 3, 0, 3, 0, 0, 2, 0, 3, ...","[0.24512714, 0.0483035, 0.0483035, 0.0483035, ...","[0.04515953, 0.04515953, 0.04515953, 0.0451595...",1551674,1551675,"[1551674, 1551674, 1551674, 1551674, 1551674, ...","[0.49999976, 0.49999976, 0.49999976, 0.4999997...","[0.86602557, 0.86602557, 0.86602557, 0.8660255...","[0.7818321, 0.7818321, 0.7818321, 0.7818321, 0...","[0.6234891, 0.6234891, 0.6234891, 0.6234891, 0...","[40, 40, 46, 47, 47, 47, 47, 47, 47, 47, 40, 4...","[0.5461206, 0.5461206, 0.66486025, 0.6831641, ...",7322346ca76333a3f57beb488486e3a7103b9b8ae5b81f...,"[0.00867827795445919, -0.3432416021823883, 0.1...","[6056, 3598, 10943, 7062, 7926, 9107, 4452, 12...",[6983],"[0.0, 25.0, 12.0]","[0.0, 0.0, 1.0]",3.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,64552,32,28,"[2439, 1, 1, 144, 1, 1393, 1, 1, 1, 5544, 1, 1...",28,"[4982, 1, 12, 446, 1071, 2921, 14, 5386, 2, 11...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[2, 1, 1, 2, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 2, ...","[2, 1, 1, 2, 1, 2, 1, 1, 1, 4, 1, 1, 1, 1, 2, ...","[-0.17000507, -0.17000507, -0.17000507, -0.170...","[51, 0, 0, 1, 0, 51, 0, 0, 0, 1, 0, 0, 0, 0, 1...","[3, 0, 0, 3, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 3, ...","[3, 0, 0, 3, 0, 3, 0, 0, 0, 2, 0, 0, 0, 0, 4, ...","[1.6796839, 0.0483035, 0.0483035, -0.6332114, ...","[2.1147482, 0.04515953, 0.04515953, 2.1147482,...",1552565,1552569,"[1552565, 1552565, 1552565, 1552565, 1552565, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-5.6425995e-07, -5.6425995e-07, -5.6425995e-0...","[-0.43388462, -0.43388462, -0.43388462, -0.433...","[-0.90096843, -0.90096843, -0.90096843, -0.900...","[31, 58, 58, 33, 12, 33, 58, 8, 58, 37, 24, 57...","[0.3306505, 0.86255425, 0.86255425, 0.38335788...",7350b2ee3456c1ce3b438fa1c0a14f79d229a3644988ec...,"[-0.002791411941871047, 0.05408860743045807, -...","[126942, 37103, 43807, 64264, 53843, 61638, 77...","[11340, 15677]","[25.0, 2.0, 5.0]","[0.0, 1.0, 1.0]",3.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,64560,59,28,"[1, 1496, 1, 380, 1, 1, 1, 158, 1, 158, 1, 1, ...",28,"[11164, 3120, 5, 912, 13, 982, 357, 494, 1, 49...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ...","[0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,

In [7]:
data1.dtypes

session_id_hash                          int64
day_index                                int64
product_sku_hash_count                   int32
product_sku_hash_list_seq               object
product_url_hash_count                   int32
product_url_hash_list_seq               object
has_been_removed_from_cart-list_seq     object
has_been_added_to_cart-list_seq         object
has_been_purchased-list_seq             object
has_been_detailed-list_seq              object
has_been_clicked-list_seq               object
event_type-list_seq                     object
product_action_filled-list_seq          object
nb_interactions-list_seq                object
category_hash-list_seq                  object
main_category-list_seq                  object
price_bucket-list_seq                   object
mean_price_hierarchy-list_seq           object
mean_price_main-list_seq                object
ts-first                                 int64
ts-last                                  int64
ts-list_seq  

In [8]:
data1['product_min'] = data1.product_sku_hash_list_seq.apply(min)
data1['product_max'] = data1.product_sku_hash_list_seq.apply(max)
data1['product_min'].min(), data1['product_max'].max()

(1, 57584)

In [9]:
#data1['product_max'] = data1.product_sku_hash_list_seq.apply(max)

In [10]:
#data1['product_min'].min(), data1['product_max'].max()

In [11]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['product_sku_hash_list_seq'])))
print (s)


2278          658
1        12020700
352          2499
2911          491
3184          472
           ...   
47269           1
44550           1
39344           2
47690           1
43823           1
Length: 49754, dtype: int64


In [ ]:
print(s.sort_values(ascending=True))

In [ ]:
data1['producturl_min'] = data1.product_url_hash_list_seq.apply(min)
data1['producturl_max'] = data1.product_url_hash_list_seq.apply(max)
data1['producturl_min'].min(), data1['producturl_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['product_url_hash_list_seq'])))
print (s.sort_values(ascending=True))

In [ ]:
data1.rename(columns={'event_type-list_seq': 'event_type_list_seq'}, inplace=True)
data1['eventtype_min'] = data1.event_type_list_seq.apply(min)
data1['eventtype_max'] = data1.event_type_list_seq.apply(max)
data1['eventtype_min'].min(), data1['eventtype_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['event_type_list_seq'])))
print (s.sort_values(ascending=True))

In [ ]:
data1.rename(columns={'category_hash-list_seq': 'category_hash_list_seq'}, inplace=True)
data1['categoryhash_min'] = data1.category_hash_list_seq.apply(min)
data1['categoryhash_max'] = data1.category_hash_list_seq.apply(max)
data1['categoryhash_min'].min(), data1['categoryhash_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['category_hash_list_seq'])))
print (s.sort_values(ascending=True))

In [ ]:
data1.rename(columns={'main_category-list_seq': 'main_category_list_seq'}, inplace=True)
data1['maincategory_min'] = data1.main_category_list_seq.apply(min)
data1['maincategory_max'] = data1.main_category_list_seq.apply(max)
data1['maincategory_min'].min(), data1['maincategory_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['main_category_list_seq'])))
print (s.sort_values(ascending=True))

In [ ]:
data1.rename(columns={'price_bucket-list_seq': 'price_bucket_list_seq'}, inplace=True)
data1['pricebucket_min'] = data1.price_bucket_list_seq.apply(min)
data1['pricebucket_max'] = data1.price_bucket_list_seq.apply(max)
data1['pricebucket_min'].min(), data1['pricebucket_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['price_bucket_list_seq'])))
print (s.sort_values(ascending=True))

In [ ]:
data1.rename(columns={'mean_price_hierarchy-list_seq': 'mean_price_hierarchy_list_seq'}, inplace=True)
data1['meanpricehierarchy_min'] = data1.mean_price_hierarchy_list_seq.apply(min)
data1['meanpricehierarchy_max'] = data1.mean_price_hierarchy_list_seq.apply(max)
data1['meanpricehierarchy_min'].min(), data1['meanpricehierarchy_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['mean_price_hierarchy_list_seq'])))
print (s.sort_values(ascending=True))

In [ ]:
data1.rename(columns={'mean_price_main-list_seq': 'mean_price_main_list_seq'}, inplace=True)
data1['meanpricemain_min'] = data1.mean_price_main_list_seq.apply(min)
data1['meanpricemain_max'] = data1.mean_price_main_list_seq.apply(max)
data1['meanpricemain_min'].min(), data1['meanpricemain_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['mean_price_main_list_seq'])))
print (s.sort_values(ascending=True))

In [ ]:
data1.rename(columns={'timestamp_hour_cos-list_seq': 'timestamp_hour_cos_list_seq'}, inplace=True)
data1['hourcos_min'] = data1.timestamp_hour_cos_list_seq.apply(min)
data1['hourcos_max'] = data1.timestamp_hour_cos_list_seq.apply(max)
data1['hourcos_min'].min(), data1['hourcos_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['timestamp_hour_cos_list_seq'])))
print (s.sort_values(ascending=True))

In [ ]:
data1.rename(columns={'timestamp_hour_sin-list_seq': 'timestamp_hour_sin_list_seq'}, inplace=True)
data1['hoursin_min'] = data1.timestamp_hour_sin_list_seq.apply(min)
data1['hoursin_max'] = data1.timestamp_hour_sin_list_seq.apply(max)
data1['hoursin_min'].min(), data1['hoursin_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['timestamp_hour_sin_list_seq'])))
print (s.sort_values(ascending=True))

In [ ]:
data1.rename(columns={'timestamp_wd_sin-list_seq': 'timestamp_wd_sin_list_seq'}, inplace=True)
data1['wdsin_min'] = data1.timestamp_wd_sin_list_seq.apply(min)
data1['wdsin_max'] = data1.timestamp_wd_sin_list_seq.apply(max)
data1['wdsin_min'].min(), data1['wdsin_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['timestamp_wd_sin_list_seq'])))
print (s.sort_values(ascending=True))

In [ ]:
data1.rename(columns={'timestamp_wd_cos-list_seq': 'timestamp_wd_cos_list_seq'}, inplace=True)
data1['wdcos_min'] = data1.timestamp_wd_cos_list_seq.apply(min)
data1['wdcos_max'] = data1.timestamp_wd_cos_list_seq.apply(max)
data1['wdcos_min'].min(), data1['wdcos_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['timestamp_wd_cos_list_seq'])))
print (s.sort_values(ascending=True))

In [ ]:
data1.rename(columns={'timestamp_age_days_norm-list_seq': 'timestamp_age_days_norm_list_seq'}, inplace=True)
data1['agedays_min'] = data1.timestamp_age_days_norm_list_seq.apply(min)
data1['agedays_max'] = data1.timestamp_age_days_norm_list_seq.apply(max)
data1['agedays_min'].min(), data1['agedays_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['timestamp_age_days_norm_list_seq'])))
print (s.sort_values(ascending=True))

In [ ]:
data1.rename(columns={'nb_interactions-list_seq': 'nb_interactions_list_seq'}, inplace=True)
data1['nbinteractions_min'] = data1.nb_interactions_list_seq.apply(min)
data1['nbinteractions_max'] = data1.nb_interactions_list_seq.apply(max)
data1['nbinteractions_min'].min(), data1['nbinteractions_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['nb_interactions_list_seq'])))
print (s.sort_values(ascending=True))

In [ ]:
data1.rename(columns={'has_been_added_to_cart-list_seq': 'has_been_added_to_cart_list_seq'}, inplace=True)
data1['addedcart_min'] = data1.has_been_added_to_cart_list_seq.apply(min)
data1['addedcart_max'] = data1.has_been_added_to_cart_list_seq.apply(max)
data1['addedcart_min'].min(), data1['addedcart_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['has_been_added_to_cart_list_seq'])))
print (s.sort_values(ascending=True))

In [ ]:
data1.rename(columns={'has_been_detailed-list_seq': 'has_been_detailed_list_seq'}, inplace=True)
data1['detailedcart_min'] = data1.has_been_detailed_list_seq.apply(min)
data1['detailedcart_max'] = data1.has_been_detailed_list_seq.apply(max)
data1['detailedcart_min'].min(), data1['detailedcart_max'].max()

In [ ]:
from  itertools import chain
from collections import Counter

#print (Counter(chain.from_iterable(df_list['listcol'])))
#Counter({'b': 2, 'a': 2, 'c': 2}

s = pd.Series(Counter(chain.from_iterable(data1['has_been_detailed_list_seq'])))
print (s.sort_values(ascending=True))

Split for train/valid/test
make directory "sessions_by_day"

In [ ]:
from transformers4rec.data.preprocessing import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(data),
                       output_dir= "/recsys/sessions_by_day",
                       partition_col='day_index',
                       timestamp_col='session_id_hash',
                       )

In [ ]:
#from transformers4rec.torch.utils.examples_utils import list_files
#list_files('/recsys/sessions_by_day')

In [ ]:
#INPUT_DIR1 = "/recsys/sessions_by_day/1"
#data1 = cudf.read_parquet(os.path.join(INPUT_DIR1, 'train.parquet'))
#data1.head()

GRU

In [12]:
CUDA_LAUNCH_BLOCKING="1"

In [13]:
import os
import glob

import torch
import transformers4rec.torch as tr

from transformers4rec.torch.ranking_metric import NDCGAt, RecallAt
from transformers4rec.torch.utils.examples_utils import wipe_memory

In [14]:
from merlin_standard_lib import Schema
SCHEMA_PATH = "schema_features.pb"
schema = Schema().from_proto_text(SCHEMA_PATH)
#schema = schema.select_by_name(['product_sku_hash_list_seq'])
#!cat $SCHEMA_PATH

In [15]:
sequence_length = 20
inputs = tr.TabularSequenceFeatures.from_schema(
        schema,
        max_sequence_length= sequence_length,
        masking = 'causal',
    )

In [16]:
d_model = 128
body = tr.SequentialBlock(
        inputs,
        tr.MLPBlock([d_model]),
        tr.Block(torch.nn.GRU(input_size=d_model, hidden_size=d_model, num_layers=1), [None, 20, d_model])
)

In [17]:
head = tr.Head(
    body,
    tr.NextItemPredictionTask(weight_tying=True, hf_format=True,
                              metrics=[NDCGAt(top_ks=[10, 20], labels_onehot=True),
                                       RecallAt(top_ks=[10, 20], labels_onehot=True)]),
)
model = tr.Model(head)

Projecting inputs of NextItemPredictionTask to'64' As weight tying requires the input dimension '128' to be equal to the item-id embedding dimension '64'


In [18]:
# import NVTabular dependencies
from transformers4rec.torch.utils.data_utils import NVTabularDataLoader

x_cat_names = ['product_sku_hash_list_seq', 'product_url_hash_list_seq','event_type-list_seq', 'category_hash-list_seq', 'main_category-list_seq', 'price_bucket-list_seq']
x_cont_names = ['mean_price_main-list_seq', 'mean_price_hierarchy-list_seq', 'timestamp_wd_sin-list_seq', 'timestamp_hour_cos-list_seq']

#x_cat_names, x_cont_names = ['product_sku_hash_list_seq'], []
               
# dictionary representing max sequence length for column
sparse_features_max = {
    fname: sequence_length
    for fname in x_cat_names + x_cont_names
}

schema = schema.select_by_name(x_cat_names + x_cont_names)

# Define a `get_dataloader` function to call in the training loop
def get_dataloader(path, batch_size=32):

    return NVTabularDataLoader.from_schema(
        schema,
        path,
        batch_size,
        max_sequence_length=sequence_length,
        sparse_names=x_cat_names + x_cont_names,
        sparse_max=sparse_features_max,
)

In [19]:
sparse_features_max

{'product_sku_hash_list_seq': 20,
 'product_url_hash_list_seq': 20,
 'event_type-list_seq': 20,
 'category_hash-list_seq': 20,
 'main_category-list_seq': 20,
 'price_bucket-list_seq': 20,
 'mean_price_main-list_seq': 20,
 'mean_price_hierarchy-list_seq': 20,
 'timestamp_wd_sin-list_seq': 20,
 'timestamp_hour_cos-list_seq': 20}

In [20]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer

#Set arguments for training
train_args = T4RecTrainingArguments(local_rank = -1,
                                    dataloader_drop_last = False,
                                    report_to = [],   #set empy list to avoig logging metrics to Weights&Biases
                                    gradient_accumulation_steps = 1,
                                    per_device_train_batch_size = 256,
                                    per_device_eval_batch_size = 32,
                                    output_dir = "/recsys/tmp",
                                    max_sequence_length=sequence_length,
                                    #data_loader_engine='nvtabular',
                                    learning_rate=0.00071,
                                    num_train_epochs=3,
                                    logging_steps=200,
                                   )

In [21]:
# Instantiate the T4Rec Trainer, which manages training and evaluation
trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema,
    compute_metrics=True,
)

In [22]:
OUTPUT_DIR = "/recsys/sessions_by_day"

In [23]:
%%time
start_time_window_index = 1
final_time_window_index = 89
for time_index in range(start_time_window_index, final_time_window_index):
    # Set data
    time_index_train = time_index
    time_index_eval = time_index + 1
    train_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_train}/train.parquet"))
    eval_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_eval}/valid.parquet"))

    # Initialize dataloaders
    trainer.train_dataloader = get_dataloader(train_paths, train_args.per_device_train_batch_size)
    trainer.eval_dataloader = get_dataloader(eval_paths, train_args.per_device_eval_batch_size)

    # Train on day related to time_index
    print('*'*20)
    print("Launch training for day %s are:" %time_index)
    print('*'*20 + '\n')
    trainer.reset_lr_scheduler()
    trainer.train()
    trainer.state.global_step +=1

    # Evaluate on the following day
    train_metrics = trainer.evaluate(metric_key_prefix='eval')
    print('*'*20)
    print("Eval results for day %s are:\t" %time_index_eval)
    print('\n' + '*'*20 + '\n')
    for key in sorted(train_metrics.keys()):
        print(" %s = %s" % (key, str(train_metrics[key])))
    wipe_memory()

***** Running training *****
  Num examples = 23552
  Num Epochs = 3
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 276


********************
Launch training for day 1 are:
********************



Step,Training Loss
200,5.294900




Training completed. Do not forget to share your model on huggingface.co/models =)




********************
Eval results for day 2 are:	

********************

 eval/loss = 4.6201348304748535
 eval/next-item/ndcg_at_10 = 0.5727645754814148
 eval/next-item/ndcg_at_20 = 0.5743767619132996
 eval/next-item/recall_at_10 = 0.5790185928344727
 eval/next-item/recall_at_20 = 0.5854483842849731
 eval_runtime = 1.2541
 eval_samples_per_second = 2373.052
 eval_steps_per_second = 74.158


...

***** Running training *****
  Num examples = 56320
  Num Epochs = 3
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 660


********************
Launch training for day 88 are:
********************



Step,Training Loss
200,3.214700
400,3.173900
600,3.149900


Saving model checkpoint to /recsys/tmp/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




********************
Eval results for day 89 are:	

********************

 eval/loss = 3.394317865371704
 eval/next-item/ndcg_at_10 = 0.6658735275268555
 eval/next-item/ndcg_at_20 = 0.6702532172203064
 eval/next-item/recall_at_10 = 0.685728907585144
 eval/next-item/recall_at_20 = 0.7031410336494446
 eval_runtime = 2.8427
 eval_samples_per_second = 2071.306
 eval_steps_per_second = 64.728
CPU times: user 5h 3min 31s, sys: 10min 58s, total: 5h 14min 29s
Wall time: 20min 8s


In [ ]:
train_paths

In [ ]:
eval_paths

In [ ]:
train_args.per_device_train_batch_size

In [ ]:
train_args.per_device_eval_batch_size

In [24]:
with open("results.txt", 'w') as f: 
    f.write('GRU accuracy results:')
    f.write('\n')
    for key, value in  model.compute_metrics().items(): 
        f.write('%s:%s\n' % (key, value.item()))

In [25]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}